In [0]:
CREATE TABLE IF NOT EXISTS gold.dim_payments (
  payment_sk BIGINT GENERATED ALWAYS AS IDENTITY,
  order_id STRING,
  payment_sequential INTEGER,
  payment_type STRING,
  payment_installments INTEGER,
  payment_value DECIMAL(10,2)
) 
USING DELTA


In [0]:
%python
payments_df = spark.read.table("silver.order_payments_cleaned")
payments_df.createOrReplaceTempView("payments_view")


In [0]:
MERGE INTO gold.dim_payments t
USING payments_view s
ON t.order_id = s.order_id

WHEN MATCHED THEN
  UPDATE SET 
    payment_sequential = s.payment_sequential,
    payment_type = s.payment_type,
    payment_installments = s.payment_installments,
    payment_value = s.payment_value
  
WHEN NOT MATCHED
  THEN INSERT (
    order_id,
    payment_sequential,
    payment_type,
    payment_installments,
    payment_value
  ) VALUES (
    s.order_id,
    s.payment_sequential,
    s.payment_type,
    s.payment_installments,
    s.payment_value
  )


In [0]:
SELECT * FROM gold.dim_payments LIMIT 10;